# Risk Analytics - Fire Perimeter Intersections

The code is a Python script that uses several libraries including Pandas, GeoPandas, and Folium to create a map of property locations and fire perimeters. It performs several steps, including:

-Loading wildfire perimeters shapefile and state boundary shapefile into geopandas dataframes

-Filtering the state boundaries shapefile to only include states of interest and reprojecting it to match the CRS of the wildfire perimeters shapefile

-Performing a spatial join to keep only fire perimeters within the states of interest

-Loading an Excel file with property locations into a pandas dataframe and converting it to a geopandas dataframe with a Point geometry column

-Performing a spatial join between the property locations and fire perimeters geodataframes to assign each property to a fire perimeter

-Grouping the property locations by fire perimeter to count the number of properties in each fire perimeter

-Merging the fire perimeter geodataframe with the property counts dataframe

-Performing another spatial join between the fire perimeters and property locations

-Counting the number of property locations per fire perimeter and sorting the fire perimeters by the count of properties in each perimeter

-Printing the counts for each fire perimeter in descending order

-Creating a dictionary to store the ObjectID and associated fire of properties within fire perimeters 

-Looping through the fire perimeters and their OBJECTID to find the associated properties and store them in the dictionary

-Creating a folium map and adding the fire perimeters and property locations to it

-Displaying the map and saving it to an HTML file.

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
import shapely
from shapely.geometry import Point

In [ ]:
# Load the wildfire perimeters shapefile into a geopandas dataframe
fire_perimeters_gdf = gpd.read_file('G:\\.shortcut-targets-by-id\\1TNkujbZ3vYE5Jr61hgbT7tHPCKankget\\Pilots\\2.0 RAC Operations\\Shapefiles\\WFIGS_-_Wildland_Fire_Perimeters_Full_History\\FH_Perimeter.shp')

In [ ]:
# Print the column names in the GeoDataFrame
print(fire_perimeters_gdf.columns)

In [ ]:
# Load state boundary shapefile into a geopandas dataframe
state_boundaries_gdf = gpd.read_file('G:\\.shortcut-targets-by-id\\1TNkujbZ3vYE5Jr61hgbT7tHPCKankget\\Pilots\\2.0 RAC Operations\\Shapefiles\\US_States\\cb_2018_us_state_500k.shp')

In [ ]:
# Print the column names in the GeoDataFrame
print(state_boundaries_gdf.columns)

In [ ]:
# Filter the state boundaries shapefile to only include the states you care about
states_of_interest = ['CA'] 
state_boundaries_gdf = state_boundaries_gdf[state_boundaries_gdf['STUSPS'].isin(states_of_interest)]

In [ ]:
# Reproject the state boundaries shapefile to match the CRS of the fire perimeters shapefile
state_boundaries_gdf = state_boundaries_gdf.to_crs(fire_perimeters_gdf.crs)

In [ ]:
# Perform a spatial join to keep only fire perimeters within states of interest
fire_perimeters_filtered_gdf = gpd.sjoin(
    fire_perimeters_gdf.to_crs(state_boundaries_gdf.crs),
    state_boundaries_gdf,
    how='inner',
    predicate='intersects',
    lsuffix='fire',
    rsuffix='state'
)

In [ ]:
print(fire_perimeters_filtered_gdf.columns)

In [ ]:
# Load the excel file with property locations into a pandas dataframe
property_locations_df = pd.read_excel('C:\\Users\RossMartin\\Desktop\\1024_travelers_zfire_final_TOP3.xlsx')

In [ ]:
# Convert the property locations dataframe to a geopandas dataframe by creating a Point geometry column
property_locations_gdf = gpd.GeoDataFrame(
    property_locations_df,
    geometry=gpd.points_from_xy(property_locations_df.Longitude, property_locations_df.Latitude),
    crs='EPSG:4326'  # specify the coordinate reference system (CRS) of the geodataframe
)

In [ ]:
# Perform a spatial join between the property locations and fire perimeters geodataframes to assign each property to a fire perimeter
property_locations_joined = gpd.sjoin(property_locations_gdf, fire_perimeters_filtered_gdf, how='left', predicate='within')

In [ ]:
# Group the property locations by fire perimeter to count the number of properties in each fire perimeter
property_locations_by_perimeter = property_locations_joined.groupby('OBJECTID')['Address'].agg(['count']).reset_index()

In [ ]:
# Merge the fire perimeter geodataframe with the property counts dataframe
fire_perimeters_count_gdf = fire_perimeters_filtered_gdf.merge(property_locations_by_perimeter, on='OBJECTID', how='left')

In [ ]:
# Perform a spatial join between the fire perimeters and property locations
joined_gdf = gpd.sjoin(fire_perimeters_filtered_gdf, property_locations_gdf, how='left', predicate='contains')

print(joined_gdf.columns)

# Count the number of property locations per fire perimeter
property_counts = joined_gdf.groupby('poly_Incid')['Latitude'].count()

# Sort the fire perimeters by the count of properties in each perimeter
property_counts_sorted = property_counts.sort_values(ascending=False)

# Print the counts for each fire perimeter in descending order
for index, count in property_counts_sorted.items():
    if count > 0:
        print(f"Fire perimeter {index} has {count} properties.")

In [ ]:
# get the ObjectID of properties within fire perimeters
properties_within_fire = property_locations_joined[property_locations_joined['poly_Incid'].notnull()]['ObjectID'].unique()

# create a dictionary to store the ObjectID and associated fire
objectid_fire_dict = {}

# loop through the fire perimeters and their OBJECTID to find the associated properties
for index, row in fire_perimeters_filtered_gdf.iterrows():
    fire = row['poly_Incid']
    properties = property_locations_joined[property_locations_joined['poly_Incid'] == fire]['ObjectID'].unique()
    for prop in properties:
        if prop in properties_within_fire:
            if prop not in objectid_fire_dict:
                objectid_fire_dict[prop] = []
            objectid_fire_dict[prop].append(fire)

# print the ObjectID and associated fires
for prop, fires in objectid_fire_dict.items():
    print(f"ObjectID {prop} was within fire(s): {', '.join(fires)}")


In [ ]:
# create the folium map
inclusion_map = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

# add the fire perimeters to the map
folium.GeoJson(fire_perimeters_count_gdf[['OBJECTID', 'poly_Incid', 'geometry', 'count']].to_json(),
               name='Fire Perimeters', 
               tooltip=folium.features.GeoJsonTooltip(fields=['poly_Incid', 'count'], aliases=['Fire', 'Property Count']),
               style_function=lambda x: {'weight': 1, 'color': 'red', 'fillOpacity': 0.1},
               highlight_function=lambda x: {'weight': 3, 'color': 'blue', 'fillOpacity': 0.2},
               ).add_to(inclusion_map)

# add the property locations to the map
for idx, row in property_locations_joined.iterrows():
    if row['poly_Incid'] in fire_perimeters_count_gdf['poly_Incid'].unique():
        folium.CircleMarker(location=[row.geometry.y, row.geometry.x], radius=.1, color='blue', fill=True, fill_color='blue').add_to(inclusion_map)

# display the map
inclusion_map

In [ ]:
# Save the map to HTML with the new file name
inclusion_map.save('wildfire_inclusion_map' + '.html')
print("Map Saved to Directory.")